# 📓 03 - Pipeline RAG básico com FAISS + LLM (OpenAI API)

Neste notebook, vamos:
✅ Recuperar contexto relevante usando busca vetorial (FAISS).  
✅ Usar um LLM (via OpenAI API) para gerar uma resposta final.  
✅ Simular um pipeline RAG simples e funcional!

---

## ⚙️ Setup inicial

In [2]:
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import faiss
from openai import OpenAI

load_dotenv()

# Inicializar cliente OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

c:\Users\vinicius\Documents\Projetos\RAG-study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 📝 1️⃣ Indexar alguns textos

In [3]:
texts = [
    "Retrieval-Augmented Generation (RAG) combina recuperação de informações e geração de linguagem natural.",
    "O sol nasce no leste e se põe no oeste.",
    "Python é uma linguagem de programação muito popular.",
    "Modelos de linguagem podem gerar respostas criativas.",
    "A IA está revolucionando diversos setores, como saúde e finanças."
]

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model.encode(texts, convert_to_numpy=True)
faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

### 🔎 2️⃣ Função de recuperação de contexto

In [4]:
def search_top_k(query, k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, k)
    
    retrieved_texts = [texts[idx] for idx in indices[0]]
    return retrieved_texts

### 🔧 3️⃣ Função para gerar resposta final usando OpenAI (RAG-like)

In [5]:
def generate_rag_response(query, retrieved_texts):
    contexto = "\n".join(retrieved_texts)
    prompt = f"""
Você é um assistente que responde perguntas com base no contexto abaixo.
Contexto:
{contexto}

Pergunta: {query}
Resposta:
"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Você é um assistente útil."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

### 🚀 4️⃣ Teste do pipeline RAG

In [6]:
query = "Como a IA está mudando as indústrias?"

# Recuperar contexto relevante
contexto = search_top_k(query, k=3)
print("🔹 Contexto recuperado:", contexto)

# Gerar resposta final
resposta = generate_rag_response(query, contexto)
print("\n🔹 Resposta final:")
print(resposta)

🔹 Contexto recuperado: ['A IA está revolucionando diversos setores, como saúde e finanças.', 'Python é uma linguagem de programação muito popular.', 'Modelos de linguagem podem gerar respostas criativas.']

🔹 Resposta final:
A IA está transformando as indústrias ao automatizar processos, aumentar a eficiência e proporcionar insights avançados a partir de grandes volumes de dados. No setor de saúde, a IA está sendo utilizada para melhorar diagnósticos, personalizar tratamentos e otimizar a gestão hospitalar. No campo das finanças, está sendo usada para detectar fraudes, melhorar a análise de risco e automatizar transações financeiras. Essa tecnologia está permitindo inovações que antes eram consideradas impossíveis, impulsionando mudanças significativas na forma como essas indústrias operam.
